In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import precision_score, recall_score, f1_score


In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [4]:
# CIFAR-10 Dataset
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True, num_workers=2)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False, num_workers=2)

100%|██████████| 170M/170M [00:05<00:00, 29.2MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
class Lenet_5(nn.Module):
  def __init__(self, num_classes=10):
    super(Lenet_5, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, kernel_size=5)
    self.pool = nn.MaxPool2d(kernel_size=2,stride=2)
    self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self,x):
    x = self.pool(torch.relu(self.conv1(x)))
    x = self.pool(torch.relu(self.conv2(x)))
    x = x.view(-1, 16*5*5)
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = self.fc3(x)
    return x




In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [6]:
def train_and_test(model, train_loader, test_loader, criterion, optimizer, num_epochs, device):
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  scheduler = StepLR(optimizer, step_size=10, gamma=0.1)

  train_loss_list = []


  for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
      images = images.to(device)
      labels = labels.to(device)
      optimizer.zero_grad()
      outputs = model(images)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()
      running_loss += loss.item()
    scheduler.step()
  # Generating Loss Curve
    train_loss = running_loss / len(train_loader)
    train_loss_list.append({'train_loss':train_loss})
    print(f'[Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}')

  # Test the model

  model.eval()
  correct = 0
  total = 0
  with torch.no_grad():
    for data in test_loader:
      images, labels = data
      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  # Calculating Accuracy
  accuracy = 100 * correct / total

  #Evaluation Metrics
  precision = precision_score(labels.cpu().numpy(), predicted.cpu().numpy(), average='macro')
  recall = recall_score(labels.cpu().numpy(), predicted.cpu().numpy(), average='macro')
  f1 = f1_score(labels.cpu().numpy(), predicted.cpu().numpy(), average='macro')

  print(f'Accuracy of the network : {accuracy:.2f}%')
  print(f'Precision: {precision:.2f}')
  print(f'Recall: {recall:.2f}')
  print(f'F1-score: {f1:.2f}')

  metrics = {'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1': f1}
  return metrics, train_loss_list

In [19]:
class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(in_channels, in_channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(in_channels // reduction, in_channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

class SENet(nn.Module):
    def __init__(self, num_classes=10):
        super(SENet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.se1 = SEBlock(64)

        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(128)
        self.se2 = SEBlock(128)

        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn3 = nn.BatchNorm2d(256)
        self.se3 = SEBlock(256)

        self.fc = nn.Linear(256 * 32 * 32, num_classes)

    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.se1(x)

        x = self.relu(self.bn2(self.conv2(x)))
        x = self.se2(x)

        x = self.relu(self.bn3(self.conv3(x)))
        x = self.se3(x)

        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


In [ ]:
class Alexnet(nn.Module):
  def __init__(self, num_classes=10):
    super(Alexnet, self).__init__()
    self.features = nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(64, 192, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(192, 384, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(384, 256, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(256, 256, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),
    )
    self.classifier = nn.Sequential(
        nn.Dropout(),
        nn.Linear(256*4*4, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(),
        nn.Linear(4096, 4096),
        nn.ReLU(inplace=True),
        nn.Linear(4096, num_classes),
    )
  def forward(self,x):
    x = self.features(x)
    x = x.view(x.size(0), 256*4*4)
    x = self.classifier(x)
    return x

In [ ]:

class VGGnet(nn.Module):
    def __init__(self, num_classes=10):
        super(VGGnet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)

        self.conv5 = nn.Conv2d(128,256, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(256*4*4, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, num_classes)

    def forward(self,x):
      x = torch.relu(self.conv1(x))
      x = torch.relu(self.conv2(x))
      x = self.pool(x)

      x = torch.relu(self.conv3(x))
      x = torch.relu(self.conv4(x))
      x = self.pool(x)

      x = torch.relu(self.conv5(x))
      x = torch.relu(self.conv6(x))
      x = self.pool(x)

      x = x.view(x.size(0), -1)
      x = torch.relu(self.fc1(x))
      x = torch.relu(self.fc2(x))
      x = self.fc3(x)
      return x

In [ ]:
import pickle
criterion = nn.CrossEntropyLoss()
model_1 = Lenet_5(num_classes=10).to(device)
optimizer = optim.Adam(model_1.parameters(), lr=0.001)
train_loss_list_1, metrics_1 = train_and_test(model_1, train_loader, test_loader, criterion, optimizer, 100, device)
pickle.dump(train_loss_list_1, open('train_loss_list_1.p', 'wb'))
pickle.dump(metrics_1, open('metrics_1.p', 'wb'))

[Epoch [1/100], Loss: 1.7293
[Epoch [2/100], Loss: 1.4275
[Epoch [3/100], Loss: 1.3077
[Epoch [4/100], Loss: 1.2268
[Epoch [5/100], Loss: 1.1674
[Epoch [6/100], Loss: 1.1177
[Epoch [7/100], Loss: 1.0741
[Epoch [8/100], Loss: 1.0362
[Epoch [9/100], Loss: 1.0017
[Epoch [10/100], Loss: 0.9722
[Epoch [11/100], Loss: 0.8922
[Epoch [12/100], Loss: 0.8786
[Epoch [13/100], Loss: 0.8732
[Epoch [14/100], Loss: 0.8673
[Epoch [15/100], Loss: 0.8636
[Epoch [16/100], Loss: 0.8586
[Epoch [17/100], Loss: 0.8548
[Epoch [18/100], Loss: 0.8514
[Epoch [19/100], Loss: 0.8475
[Epoch [20/100], Loss: 0.8433
[Epoch [21/100], Loss: 0.8323
[Epoch [22/100], Loss: 0.8311
[Epoch [23/100], Loss: 0.8304
[Epoch [24/100], Loss: 0.8301
[Epoch [25/100], Loss: 0.8296
[Epoch [26/100], Loss: 0.8294
[Epoch [27/100], Loss: 0.8291
[Epoch [28/100], Loss: 0.8283
[Epoch [29/100], Loss: 0.8280
[Epoch [30/100], Loss: 0.8278
[Epoch [31/100], Loss: 0.8264
[Epoch [32/100], Loss: 0.8263
[Epoch [33/100], Loss: 0.8264
[Epoch [34/100], Lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pickle
criterion = nn.CrossEntropyLoss()
model_2 = VGGnet(num_classes=10).to(device)
optimizer = optim.Adam(model_2.parameters(), lr=0.001)
train_loss_list_2, metrics_2 = train_and_test(model_2, train_loader, test_loader, criterion, optimizer, 100, device)
pickle.dump(train_loss_list_2, open('train_loss_list_2.p', 'wb'))
pickle.dump(metrics_2, open('metrics_2.p', 'wb'))


[Epoch [1/100], Loss: 1.6409
[Epoch [2/100], Loss: 1.0901
[Epoch [3/100], Loss: 0.8161
[Epoch [4/100], Loss: 0.6431
[Epoch [5/100], Loss: 0.5137
[Epoch [6/100], Loss: 0.4050
[Epoch [7/100], Loss: 0.3140
[Epoch [8/100], Loss: 0.2365
[Epoch [9/100], Loss: 0.1805
[Epoch [10/100], Loss: 0.1453
[Epoch [11/100], Loss: 0.0407
[Epoch [12/100], Loss: 0.0124
[Epoch [13/100], Loss: 0.0056
[Epoch [14/100], Loss: 0.0029
[Epoch [15/100], Loss: 0.0016
[Epoch [16/100], Loss: 0.0009
[Epoch [17/100], Loss: 0.0005
[Epoch [18/100], Loss: 0.0004
[Epoch [19/100], Loss: 0.0002
[Epoch [20/100], Loss: 0.0002
[Epoch [21/100], Loss: 0.0001
[Epoch [22/100], Loss: 0.0001
[Epoch [23/100], Loss: 0.0001
[Epoch [24/100], Loss: 0.0001
[Epoch [25/100], Loss: 0.0001
[Epoch [26/100], Loss: 0.0001
[Epoch [27/100], Loss: 0.0001
[Epoch [28/100], Loss: 0.0001
[Epoch [29/100], Loss: 0.0001
[Epoch [30/100], Loss: 0.0001
[Epoch [31/100], Loss: 0.0001
[Epoch [32/100], Loss: 0.0001
[Epoch [33/100], Loss: 0.0001
[Epoch [34/100], Lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pickle
criterion = nn.CrossEntropyLoss()
model_3 = Alexnet(num_classes=10).to(device)
optimizer = optim.Adam(model_3.parameters(), lr=0.001)
train_loss_list_3, metrics_3 = train_and_test(model_3, train_loader, test_loader, criterion, optimizer, 100, device)
pickle.dump(train_loss_list_3, open('train_loss_list_3.p', 'wb'))
pickle.dump(metrics_3, open('metrics_3.p', 'wb'))


[Epoch [1/100], Loss: 1.7480
[Epoch [2/100], Loss: 1.2846
[Epoch [3/100], Loss: 1.0850
[Epoch [4/100], Loss: 0.9527
[Epoch [5/100], Loss: 0.8498
[Epoch [6/100], Loss: 0.7680
[Epoch [7/100], Loss: 0.7018
[Epoch [8/100], Loss: 0.6574
[Epoch [9/100], Loss: 0.6081
[Epoch [10/100], Loss: 0.5569
[Epoch [11/100], Loss: 0.4008
[Epoch [12/100], Loss: 0.3583
[Epoch [13/100], Loss: 0.3329
[Epoch [14/100], Loss: 0.3113
[Epoch [15/100], Loss: 0.2951
[Epoch [16/100], Loss: 0.2810
[Epoch [17/100], Loss: 0.2657
[Epoch [18/100], Loss: 0.2488
[Epoch [19/100], Loss: 0.2417
[Epoch [20/100], Loss: 0.2292
[Epoch [21/100], Loss: 0.2037
[Epoch [22/100], Loss: 0.2005
[Epoch [23/100], Loss: 0.1976
[Epoch [24/100], Loss: 0.1954
[Epoch [25/100], Loss: 0.1922
[Epoch [26/100], Loss: 0.1939
[Epoch [27/100], Loss: 0.1886
[Epoch [28/100], Loss: 0.1880
[Epoch [29/100], Loss: 0.1851
[Epoch [30/100], Loss: 0.1857
[Epoch [31/100], Loss: 0.1861
[Epoch [32/100], Loss: 0.1839
[Epoch [33/100], Loss: 0.1828
[Epoch [34/100], Lo

In [21]:
import pickle
criterion = nn.CrossEntropyLoss()
model_4 = SENet(num_classes=10).to(device)
optimizer = optim.Adam(model_4.parameters(), lr=0.001)
train_loss_list_4, metrics_4 = train_and_test(model_4, train_loader, test_loader, criterion, optimizer, 100, device)
pickle.dump(train_loss_list_4, open('train_loss_list_4.p', 'wb'))
pickle.dump(metrics_4, open('metrics_4.p', 'wb'))


[Epoch [1/100], Loss: 3.1891
[Epoch [2/100], Loss: 1.0763
[Epoch [3/100], Loss: 0.8809
[Epoch [4/100], Loss: 0.7428
[Epoch [5/100], Loss: 0.6414
[Epoch [6/100], Loss: 0.5153
[Epoch [7/100], Loss: 0.4118
[Epoch [8/100], Loss: 0.3070
[Epoch [9/100], Loss: 0.2070
[Epoch [10/100], Loss: 0.1585
[Epoch [11/100], Loss: 0.0528
[Epoch [12/100], Loss: 0.0215
[Epoch [13/100], Loss: 0.0128
[Epoch [14/100], Loss: 0.0096
[Epoch [15/100], Loss: 0.0063
[Epoch [16/100], Loss: 0.0047
[Epoch [17/100], Loss: 0.0037
[Epoch [18/100], Loss: 0.0031
[Epoch [19/100], Loss: 0.0024
[Epoch [20/100], Loss: 0.0028
[Epoch [21/100], Loss: 0.0021
[Epoch [22/100], Loss: 0.0015
[Epoch [23/100], Loss: 0.0013
[Epoch [24/100], Loss: 0.0015
[Epoch [25/100], Loss: 0.0011
[Epoch [26/100], Loss: 0.0012
[Epoch [27/100], Loss: 0.0010
[Epoch [28/100], Loss: 0.0011
[Epoch [29/100], Loss: 0.0010
[Epoch [30/100], Loss: 0.0009
[Epoch [31/100], Loss: 0.0008
[Epoch [32/100], Loss: 0.0007
[Epoch [33/100], Loss: 0.0007
[Epoch [34/100], Lo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
